In [1]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# ------------------------------
# ENTER YOUR PARAMETERS HERE:
ARUCO_DICT = cv.aruco.DICT_6X6_100
SQUARES_VERTICALLY = 7
SQUARES_HORIZONTALLY = 5
SQUARE_LENGTH = 0.03
MARKER_LENGTH = 0.015
LENGTH_PX = 640   # total length of the page in pixels
MARGIN_PX = 20    # size of the margin in pixels
SAVE_NAME = 'ChArUco_Marker.png'
# ------------------------------

In [3]:
def create_and_save_new_board(filename=SAVE_NAME, number=0):
    
    dictionary = cv.aruco.getPredefinedDictionary(ARUCO_DICT)
    board = cv.aruco.CharucoBoard((SQUARES_VERTICALLY, SQUARES_HORIZONTALLY), SQUARE_LENGTH, MARKER_LENGTH, dictionary)
    size_ratio = SQUARES_HORIZONTALLY / SQUARES_VERTICALLY
    img = cv.aruco.CharucoBoard.generateImage(board, (LENGTH_PX, int(LENGTH_PX / 2**0.5)), marginSize=MARGIN_PX)
    cv.imwrite(f'data/charuco/{filename}_{number}.png', img)

In [4]:
PATH_TO_YOUR_IMAGES = 'data/calibration_images/'  # Change this to your path

In [11]:
def calibrate_and_save_parameters():
    # Define the aruco dictionary and charuco board
    dictionary = cv.aruco.getPredefinedDictionary(ARUCO_DICT)
    board = cv.aruco.CharucoBoard((SQUARES_VERTICALLY, SQUARES_HORIZONTALLY), SQUARE_LENGTH, MARKER_LENGTH, dictionary)
    params = cv.aruco.DetectorParameters()
    image_size = None
    
    # Load PNG images from folder
    image_files = [os.path.join(PATH_TO_YOUR_IMAGES, f) for f in os.listdir(PATH_TO_YOUR_IMAGES) if f.endswith(".JPG")]
    image_files.sort()  # Ensure files are in order

    all_charuco_corners = []
    all_charuco_ids = []

    for image_file in image_files:
        image = cv.imread(image_file)
        if image_size is None:
            image_size = image.shape[:2]
            
        image_copy = image.copy()
        marker_corners, marker_ids, _ = cv.aruco.detectMarkers(image, dictionary, parameters=params)
        
        # If at least one marker is detected
        if len(marker_ids) > 0:
            cv.aruco.drawDetectedMarkers(image_copy, marker_corners, marker_ids)
            charuco_retval, charuco_corners, charuco_ids = cv.aruco.interpolateCornersCharuco(marker_corners, marker_ids, image, board)
            if charuco_retval:
                all_charuco_corners.append(charuco_corners)
                all_charuco_ids.append(charuco_ids)

    # Calibrate camera
    retval, camera_matrix, dist_coeffs, rvecs, tvecs = cv.aruco.calibrateCameraCharuco(all_charuco_corners, all_charuco_ids, board, image_size, None, None)

    # Save calibration data
    np.save(PATH_TO_YOUR_IMAGES + 'camera_matrix.npy', camera_matrix)
    np.save(PATH_TO_YOUR_IMAGES + 'dist_coeffs.npy', dist_coeffs)

In [12]:
calibrate_and_save_parameters()

In [14]:
camera_matrix = np.load(PATH_TO_YOUR_IMAGES + 'camera_matrix.npy')
dist_coeffs = np.load(PATH_TO_YOUR_IMAGES + 'dist_coeffs.npy')

In [15]:
def calibrate_and_save_img(image_file, camera_matrix, dist_coeffs):
    image = cv.imread(image_file)
    undistorted_image = cv.undistort(image, camera_matrix, dist_coeffs)
    cv.imwrite(os.path.join('data', os.path.basename(image_file).split('.')[0] + '_undistorted.jpg'), undistorted_image)

In [16]:
calibrate_and_save_img(PATH_TO_YOUR_IMAGES + '_DSC3881.JPG', camera_matrix, dist_coeffs)